In [7]:
!pip install langchain-groq

  Using cached langchain_groq-0.3.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached groq-0.22.0-py3-none-any.whl.metadata (15 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached langchain_groq-0.3.2-py3-none-any.whl (15 kB)
Using cached groq-0.22.0-py3-none-any.whl (126 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("faiss_index", embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),allow_dangerous_deserialization=True)

In [22]:
retriever = db.as_retriever(search_type='similarity',search_kwargs={"k":4})

In [12]:
from langchain_groq import ChatGroq

model = ChatGroq(model_name='llama-3.3-70b-versatile')

In [17]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a PDF Reader Assistant."
    "Provide concise and professional responses."
    "If the user asks a part of PDF, give clear and helpful information about it."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate(
    [
        ("system",system_prompt),
        ("human", "{input}"),
    ]
    )

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain

create_stuff_documents_chain = create_stuff_documents_chain(model, prompt)

In [23]:
from langchain.chains import create_retrieval_chain

retrieval = create_retrieval_chain(retriever, create_stuff_documents_chain)

In [25]:
result = retrieval.invoke({"input":"Hi What is simbolo"})
result

{'input': 'Hi What is simbolo',
 'context': [Document(id='1acf6f90-6172-4041-a86d-e7002b8e41a5', metadata={'source': 'Simbolo Information - Sheet1.csv', 'row': 0}, page_content='Questions: What is Simbolo?\nAnswers: Símbolo is a training school which provides high-quality education in science and technology by offering courses in Artificial Intelligence, Data Science, Web Development, Mobile Development, and Information Technology. These courses combine theoretical knowledge with practical, project-based learning.'),
  Document(id='992861b5-dbce-4d13-b10c-e617f5116c64', metadata={'source': 'Simbolo Information - Sheet1.csv', 'row': 15}, page_content='Questions: Where can we contact the Simbolo?\nAnswers: You can contact Simbolo in the following ways:\nEmail: simboloit@gmail.com\nPhone: +959951716847\nMessenger: https://m.me/Simboloit'),
  Document(id='93d8db19-4aca-4f34-a8f1-4839328e4ab2', metadata={'source': 'Simbolo Information - Sheet1.csv', 'row': 14}, page_content='Questions: What

In [26]:
result['answer']

'Símbolo is a training school that provides high-quality education in science and technology, offering courses in Artificial Intelligence, Data Science, Web Development, Mobile Development, and Information Technology, combining theoretical knowledge with practical, project-based learning.'